# Prerequisites

## 개요

AWS 비디오 이해 워크숍에 오신 것을 환영합니다!

이 워크숍은 두 가지 주요 부분으로 구성되어 있습니다: 1. Bedrock Data Automation을 사용한 미디어 분석과 2. Amazon Nova를 사용한 미디어 분석. 각 섹션에 대한 개요를 살펴보세요. 이 노트북을 실행한 후에는 파트 1과 파트 2를 독립적으로 실행할 수 있습니다.

**사전 요구사항**

메인 워크숍을 실행하기 전에, 이 노트북을 사용하여 노트북 환경을 설정하게 됩니다.

**파트 1: Bedrock Data Automation(BDA)을 사용한 미디어 분석:**

이 섹션의 노트북들은 BDA API와 사용 사례에 대한 개요를 제공합니다. 순서에 관계없이 실행할 수 있습니다.

1. [BDA를 사용한 영화 추출 및 분석](1-media-analysis-using-bda/01-extract-analyze-a-movie.ipynb)
2. [문맥 기반 광고 오버레이](1-media-analysis-using-bda/02-contextual-ad-overlay.ipynb)

**파트 2: Amazon Nova를 사용한 미디어 분석:**

기초 노트북에서는 노트북 환경을 설정하고, 샘플 비디오를 클립으로 분할하여 준비하며, Foundation 모델을 사용하여 비디오 클립에 대한 인사이트를 생성하는 실험을 하게 됩니다. 워크숍의 두 번째 부분에서는 이러한 기초를 바탕으로 다양한 비디오 이해 사용 사례를 해결합니다. 사용 사례들은 독립적이며 순서에 관계없이 실행할 수 있습니다.

**기초 과정 (사용 사례 실행 전 필수)**
1. [시각적 비디오 세그먼트: 프레임, 샷, 장면](2-media-analysis-using-amazon-nova/01A-visual-segments-frames-shots-scenes.ipynb) (20분)
2. [오디오 세그먼트](2-media-analysis-using-amazon-nova/01B-audio-segments.ipynb) (10분)

**사용 사례 (선택사항, 순서 무관):**

기초 노트북을 실행한 후에는 원하는 사용 사례를 선택할 수 있습니다. AWS 워크숍 이벤트에 참여하시는 경우, 2시간 세션 동안 기초 과정과 하나의 사용 사례를 완료할 수 있습니다:

* [광고 중단 감지 및 문맥 기반 광고 타겟팅](2-media-analysis-using-amazon-nova/02-ad-breaks-and-contextual-ad-targeting.ipynb) (20분) - 광고 삽입 기회를 식별합니다. 표준 분류법을 사용하여 비디오 콘텐츠를 광고 콘텐츠와 매칭합니다.
* [비디오 요약](2-media-analysis-using-amazon-nova/03-video-summarization.ipynb) (20분) - 긴 비디오에서 짧은 형식의 비디오를 생성합니다
* [의미론적 비디오 검색](2-media-analysis-using-amazon-nova/04-semantic-video-search.ipynb) (20분) - 이미지와 자연어를 사용하여 관련 클립을 찾기 위한 비디오 검색

**리소스**

이 워크숍의 활동들은 AWS 솔루션 가이드를 기반으로 합니다. [추가 리소스](./09-resources.ipynb) 랩에는 관련 참조 아키텍처, 코드 샘플 및 블로그 게시물에 대한 링크가 포함되어 있습니다.

# 파이썬 패키지와 ffmpeg 설치

- ffmpeg: 비디오 및 이미지 처리용
- faiss: 벡터 저장소용
- webvtt-py: 자막 파일 파싱용
- termcolor: 출력 형식 지정용

In [4]:
## install ffmpeg
!sudo apt update -y && sudo apt-get -y install ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
2 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
## Make sure ffmpeg is installed
!which ffmpeg

/usr/bin/ffmpeg


In [3]:
%pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 165.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 185.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 168.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 162.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 177.7 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110795 sha256=b8dcc70eacfde97b700926f21d17c643f204d07c186a453eaad9e96807ff3dd8
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/df/ba/4b/0917fc0c8833c8ba7016565fc975b74c67bc8610806e930272
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.2.1
    Uninstalling decorator-5.2.1:
      Successfully uninstalled decorator-5.2.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.1
    

## SageMaker 기본 리소스 얻기

In [5]:
import sagemaker

sagemaker_resources = {}
#sagemaker_resources["session"] = sagemaker.Session()
sagemaker_resources["role"] = sagemaker.get_execution_role()
sagemaker_resources["region"] = sagemaker.Session()._region_name

print(sagemaker_resources)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
{'role': 'arn:aws:iam::461688981841:role/workshop-SageMakerExecutionRole-RVcj7OLUoSnS', 'region': 'us-west-2'}


# AWS 세션 리소스 설정

AWS 호스팅 이벤트에서 실행하는 경우, AWS 리소스가 사전에 생성되어 있으며 아래 셀을 사용하여 CloudFormation 출력에서 해당 리소스들을 로드하게 됩니다. 자체 계정에서 실행하는 경우, 수동으로 생성한 리소스에서 다음 변수들을 설정해야 합니다.

## CloudFormation 스택 출력 가져오기 (AWS 호스팅 이벤트 전용)

In [6]:
import boto3
from IPython.display import JSON
cf = boto3.client(service_name="cloudformation")
stack = response = cf.describe_stacks(StackName='workshop')

In [7]:
JSON(stack)

<IPython.core.display.JSON object>

In [8]:
session = {}
session['bucket'] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "S3BucketName")
session['MediaConvertRole'] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "MediaConvertRole")
session["AOSSCollectionEndpoint"] = next(item["OutputValue"] for item in stack['Stacks'][0]['Outputs'] if item["OutputKey"] == "AOSSCollectionEndpoint")

# Amazon Q Developer 찾기

SageMaker Studio의 Jupyter 노트북에는 Amazon Q Developer가 활성화되어 있습니다.

1. Q Developer를 사용하려면 왼쪽 사이드바 메뉴에서 Q Developer 채팅 아이콘을 클릭하세요. 활성화된 사이드 패널이 이제 Amazon Q Developer가 됩니다.
<br></br>
<img src="static/images/00-qdev-sidebar1.png" alt="Q Developer Sidebar" style="width: 600px;"/>
<br></br>
2. 질문을 해보면서 사용해보세요. 예를 들어, 다음과 같이 물어볼 수 있습니다: `Q Developer는 어떤 종류의 질문에 답변할 수 있나요? 간단히 설명해주세요.` 다음과 같은 응답을 받을 수 있습니다:
<br></br>
<img src="static/images/00-qdev-skills1.png" alt="Q Developer Skills" style="width: 600px;"/>
<br></br>

Throughout this workshop, you can use Q when you encounter errors or have questions about the code.  

# 다른 노트북에서 사용할 변수 저장하기

다음 실습에서 이 데이터를 사용할 예정입니다. 이 데이터를 사용하기 위해, 후속 노트북에서 사용할 수 있도록 이러한 변수들을 저장하겠습니다.

In [9]:
%store sagemaker_resources
%store session

Stored 'sagemaker_resources' (dict)
Stored 'session' (dict)


# 워크샵의 다음 섹션으로 계속하기

여기에서 Bedrock Data Automation으로 작업하는 Part1 또는 Amazon Nova 워크플로우로 작업하는 Part2 중 선택하여 진행할 수 있습니다.

1. [Part1 시작: BDA를 사용한 영화 추출 및 분석](1-media-analysis-using-bda/01-extract-analyze-a-movie.ipynb)
2. [Part2 시작: 시각적 비디오 세그먼트: 프레임, 샷 및 장면](2-media-analysis-using-amazon-nova/01A-visual-segments-frames-shots-scenes.ipynb)